<a href="https://colab.research.google.com/github/Severin-R/AIDrumDemonstrator/blob/main/Demonstrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Loading...

In [ ]:
# @title Git LFS
#@markdown Verwendung von Git LFS zum Klonen des Repository (könnte eingeschränkt funktionieren wegen Speicherrestriktionen)

#@markdown falls Git LFS nicht funktioniert mit Repository aus folgendem [Link](https://drive.google.com/drive/folders/1Mib5_X7nEbh1PeieuOlB0DYTogm9_d1e?usp=sharing) herunterladen und in eigenen Drive einbinden

!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs clone https://github.com/Severin-R/AIDrumDemonstrator.git
%cd "/content/AIDrumDemonstrator"
!git lfs pull

import os
os.chdir("/content/AIDrumDemonstrator")

In [ ]:
#@title Google Drive
#@markdown Hier bitte den Pfad zu dem Repository im eigenen Drive angeben
path = '/content/drive/MyDrive/AIDrumDemonstrator'  #@param {type: "string"}

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir(path)

# Demonstrator

In [ ]:
# @title Requirements
%%capture
!pip install ipywidgets
!pip install pytorch-lightning
!pip install pydub
!pip install pyroomacoustics


In [ ]:
# @title Start Demonstrator
from demonstrator import start_demonstrator

start_demonstrator()


Instructions for updating:
Colocations handled automatically by placer.


# Metriken

Nur möglich mit Google Colab, da externer Datensatz benötigt wird

In [ ]:
# @title Requirements
%%capture
!pip install frechet_audio_distance
!pip install torchlibrosa
!pip install pytorch-lightning

In [ ]:
# @title Select Model and generate Samples
#@markdown Auswahl des Modells in der Dropdown


select_model = "WaveGAN" #@param ["WaveGAN", "CRASH", "Neural_Granular"] {allow-input: true}
#@markdown ---

import tensorflow as tf
import os
import numpy as np
import time
from my_models import WaveGAN_Model, Crash_Model, Granular_Synthesis_Model
if path:
  os.chdir(path)
from my_metrics import IS, FAD, NDB, write_audio_files

waveGAN = WaveGAN_Model()
crash = Crash_Model()
granular = Granular_Synthesis_Model()

option_to_models = {
    "Neural_Granular": granular,
    "WaveGAN": waveGAN,
    "CRASH (very slow)": crash,
}

my_model = select_model

NUM_SAMPLES = 12000
factor = 120
batches = int(NUM_SAMPLES/factor)

generated_sounds = []

start_time = time.time()
for x in range(factor):
  generated_sounds.append(my_model.generate_n_samples(batches))

generated_sounds = np.array(generated_sounds)
generated_sounds = generated_sounds.reshape(-1, generated_sounds.shape[-1])

end_time = time.time()

print(f"Benötigte Zeit zum Generieren von {NUM_SAMPLES}: {end_time-start_time}s")

incept = IS()
fad = FAD()
ndb = NDB()
write_audio_files(generated_sounds, "/content/Temp1")



In [ ]:
# @title Calculate Metrics
#@markdown Nur möglich wenn Google Colab verwendet wird (es wird ein Referenzdatensatz benötigt)

print(f"Calculating metrics for {my_model.name} with {NUM_SAMPLES} Samples")
inception_score = incept.calculate_from_path("","/content/Temp1")
print(f"Inception Score: {inception_score}")
fad_score = fad.calculate_from_path(os.path.join(base_path,"Processed_Data_no_classes"), "/content/Temp1")
print(f"Frechet Audio Distance: {fad_score}")
ndb_score = ndb.calculate_from_path(os.path.join(base_path, "Processed_Data_no_classes"), "/content/Temp1", n_bins=12)
print(f"Number of statistically different bins with k = 12: {ndb_score}")
ndb_score = ndb.calculate_from_path(os.path.join(base_path, "Processed_Data_no_classes"), "/content/Temp1", n_bins=50)
print(f"Number of statistically different bins with k = 50: {ndb_score}")
